<b>Exercise 7: Excitatory synaptic connection between two neurons.</b><br><br>
We continue with the four-compartment model and the synapse model of exercise 5. The activation time of the synapse is now a dynamical variable, namely, the time of a spike in another, identical four-compartment neuron.

In [1]:
#Import the python libraries
import matplotlib.pyplot as plt
plt.ion()
from neuron import h
import numpy
import time
from ipywidgets import widgets, fixed, Layout


In [2]:
%%html
<style type='text/css'>
.widget-inline-hbox .widget-label {
      max-width: 250px;
      min-width: 250px;
}
</style>

In [3]:
#An auxiliary function for drawing an arrow
def drawarrow(ax,x,y,acoeff=1,prc=0.95,lw=1,lc='#000000'):
    d = [x[1]-x[0], y[1]-y[0]]; k = numpy.sqrt(d[0]**2 + d[1]**2); d = d/k
    dperp = [acoeff*z for z in [-d[1], d[0]]]; lens = k-k*(1-prc); perplen = 0.5*k*(1-prc);
    px = [x[0],x[1],x[0]+lens*d[0]+perplen*dperp[0],x[0]+lens*d[0]-perplen*dperp[0]];
    py = [y[0],y[1],y[0]+lens*d[1]+perplen*dperp[1],y[0]+lens*d[1]-perplen*dperp[1]];
    px = [px[0],px[1],px[2],px[1],px[3]]
    py = [py[0],py[1],py[2],py[1],py[3]]
    ax.plot(px,py,'k-',linewidth=lw,color=lc)


#### Run NEURON with Python interface
def runneuron(isadaptive=True,amp=0.12,dur=10,amp2=1.0,stimcomp=1,stimx=0.5,tau1=0.5,tau2=20.0,glsoma=0.03,gldend=0.03,Ra=100,cm=1.0):
    my_start_time = time.time()
    h.load_file("stdlib.hoc")                        #A default NEURON library
    h.load_file("stdrun.hoc")                        #A default NEURON library

    h('objref cvode')                                #Define an object cvode
    h('cvode = new CVode()')                         #Make cvode a time step integrator object
    h('cvode.active('+str(isadaptive*1)+')')         #Set the variable time step integration on/off

    h('create soma, dend[3]')                        #Create soma and three dendritic sections

    h.dend[0].connect(h.soma,1,0)                    #Connect the sections to each other
    h.dend[1].connect(h.soma,1,0)
    h.dend[2].connect(h.dend[0],1,0)

    h('access soma')                                 #Make soma the currently accessed section

    h('create soma2, dend2[3]')                      #Create soma and three dendritic sections
    h.dend2[0].connect(h.soma2,1,0)                  #Connect the sections to each other
    h.dend2[1].connect(h.soma2,1,0)
    h.dend2[2].connect(h.dend2[0],1,0)
    stimtextdict = {0: h.soma2, 1: h.dend2[0], 2: h.dend2[1], 3: h.dend2[2]}
    stimnamedict = {0: 'soma2', 1: 'dend2[0]', 2: 'dend2[1]', 3: 'dend2[2]'}

    for sec in [h.dend[0], h.dend[1], h.dend[2], h.dend2[0], h.dend2[1], h.dend2[2]]:
        sec.insert('pas')                            #Insert passive membrane mechanism to all sections
        sec.nseg = 5                                 #and use 5 sub-compartments for each section
    for sec in [h.soma, h.soma2]:
        sec.insert('hh')                              #Insert the Hodgkin-Huxley mechanism (includes leak) to soma
        sec.nseg = 5                                  

    #Set the passive parameters for each section:
    h.soma.Ra = Ra; h.soma.cm = cm; h.soma.diam = 10; h.soma.L = 20; h.soma.gl_hh = 0.001*glsoma
    h.dend[0].Ra = Ra; h.dend[0].cm = cm; h.dend[0].diam = 5; h.dend[0].L = 200; h.dend[0].e_pas = -54.3; h.dend[0].g_pas = 0.001*gldend
    h.dend[1].Ra = Ra; h.dend[1].cm = cm; h.dend[1].diam = 2; h.dend[1].L = 100; h.dend[1].e_pas = -54.3; h.dend[1].g_pas = 0.001*gldend
    h.dend[2].Ra = Ra; h.dend[2].cm = cm; h.dend[2].diam = 2; h.dend[2].L = 160; h.dend[2].e_pas = -54.3; h.dend[2].g_pas = 0.001*gldend
    h.soma2.Ra = Ra; h.soma2.cm = cm; h.soma2.diam = 10; h.soma2.L = 20; h.soma2.gl_hh = 0.001*glsoma
    h.dend2[0].Ra = Ra; h.dend2[0].cm = cm; h.dend2[0].diam = 5; h.dend2[0].L = 200; h.dend2[0].e_pas = -54.3; h.dend2[0].g_pas = 0.001*gldend
    h.dend2[1].Ra = Ra; h.dend2[1].cm = cm; h.dend2[1].diam = 2; h.dend2[1].L = 100; h.dend2[1].e_pas = -54.3; h.dend2[1].g_pas = 0.001*gldend
    h.dend2[2].Ra = Ra; h.dend2[2].cm = cm; h.dend2[2].diam = 2; h.dend2[2].L = 160; h.dend2[2].e_pas = -54.3; h.dend2[2].g_pas = 0.001*gldend

    h.dt = 0.025                                     #Set the time step to 0.025 ms
    h.tstop = 300                                    #Continue the simulation until 200 ms

    h('objref stim, syn1, Vrec')                     #Declare stimulation objects and Vrec

    h.stim = h.IClamp(0.5, sec = h.soma)             #Make stim an IClamp object, stimulating center (0.5) of soma
    h.stim.delay = 100                               #Stimulation starts at 100 ms
    h.stim.dur = dur                                 #and lasts 10 ms (by default)
    h.stim.amp = amp                                 #and has a given amplitude (default 0.075 nA)

    #Make syn1 an Exp2Syn object, stimulating at a given location of a given compartment
    h(stimnamedict[stimcomp]+' syn1 = new Exp2Syn('+str(stimx)+')')
    h.syn1.tau1 = tau1                               #Synaptic stimulation has given rise and
    h.syn1.tau2 = tau2                               #decay time constants
    h.syn1.e = 0                                     #Reversal potential 0 mV (glutamatergic)
    h("""
objref nc, nilstim, fih                              //Declare objects nc and nilstim and fih
soma nc = new NetCon(&v(0.5), syn1)                  //Make nc a NetCon object that activates syn1 whenever soma.v(0.5) crosses a threshold
nc.threshold = -30                                   //set threshold to -25 mV
nc.weight = """+str(0.001*amp2)+"""                  //Maximal synaptic conductance amp (default 1 nS (0.001 uS))                          
//fih = new FInitializeHandler("nc.event(100)")      //Make fih an event handler that makes syn1 activate at 100 ms always at initialization
""")

    Vrec = h.Vector()                                #Record somatic membrane potentials and time
    Vrec.record(h.soma(0.5)._ref_v, sec=h.soma)
    Vrec2 = h.Vector()                               
    Vrec2.record(h.soma2(0.5)._ref_v, sec=h.soma)
    trec = h.Vector()
    trec.record(h._ref_t)
    
    h.init()                                         #Initialize state variables
    h.run()                                          #Run the simulations

    plt.close("all")
    f, axs = plt.subplots(2, 1, sharey=False)
    axs[0].plot(trec,Vrec)
    axs[1].plot(trec,Vrec2)
    for i in range(0,2):
        axs[i].set_xlim([0, 300])
        axs[i].set_ylim([-80, 40])
        axs[i].set_ylabel('$V_m(t)$ (mV)')
    axs[0].set_xticklabels([])
    axs[0].set_title('$V_{soma}$ (presynaptic neuron)')
    axs[1].set_title('$V_{soma}$ (postsynaptic neuron)')
    axnew = f.add_axes([0.92,0.1,0.075,0.8])
    axnew.set_xlim([-2.2,18.2])
    axnew.set_ylim([-10,390])
    axnew.plot([0,0],[0,20],'b-',color='#6666FF',linewidth=6)
    axnew.plot([-2,-2],[20,100+20],'c-',linewidth=3)
    axnew.plot([2,2],[20,200+20],'g-',linewidth=1.2)
    axnew.plot([2,2],[200+20,160+200+20],'y-',color='#AABB00',linewidth=1.2)
    axnew.plot(0,0+20*0.5, 'rx')
    axnew.plot([16,16],[0,20],'b-',color='#6666FF',linewidth=6)
    axnew.plot([16-2,16-2],[20,100+20],'c-',linewidth=3)
    axnew.plot([16+2,16+2],[20,200+20],'g-',linewidth=1.2)
    axnew.plot([16+2,16+2],[200+20,160+200+20],'y-',color='#AABB00',linewidth=1.2)
    axnew.plot(16+0*(stimcomp==0)+2*(stimcomp==1)-2*(stimcomp==2)+2*(stimcomp==3),
               (0*(stimcomp==0)+20*(stimcomp==1 or stimcomp==2)+220*(stimcomp==3))+
               (20*(stimcomp==0)+200*(stimcomp==1)+100*(stimcomp==2)+160*(stimcomp==3))*stimx, 'kx')
    drawarrow(axnew,[0,16+0*(stimcomp==0)+2*(stimcomp==1)-2*(stimcomp==2)+2*(stimcomp==3)],
                 [0+20*0.5, (0*(stimcomp==0)+20*(stimcomp==1 or stimcomp==2)+220*(stimcomp==3))+
               (20*(stimcomp==0)+200*(stimcomp==1)+100*(stimcomp==2)+160*(stimcomp==3))*stimx])
    
    axnew.axis('off')
    axnew.axis('equal')
    plt.suptitle("Simulation run in "+str(time.time() - my_start_time)+" seconds.")
    plt.show()



In [4]:
mystyle = {'description_width': '200px'}
slider = widgets.interact(runneuron, isadaptive=fixed(True),
                          amp=widgets.FloatSlider(min=0.0,max=2.0,step=0.01,value=0.12,description='Amplitude of stimulus: amp',layout=Layout(width='50%'), continuous_update = False), 
                          dur=widgets.FloatSlider(min=0.0,max=100.0,step=1.0,value=10,description='Duration of stimulus: dur',layout=Layout(width='50%'), continuous_update = False), 
                          amp2=widgets.FloatSlider(min=0.0,max=30,step=0.1,value=1,description='Synaptic weight: amp2',layout=Layout(width='50%'), continuous_update = False), 
                          stimcomp=widgets.Dropdown(options={'soma': 0,'dend[0]': 1,'dend[1]': 2,'dend[2]': 3},description='Compartment with synapse: stimcomp',layout=Layout(width='50%')), 
                          stimx=widgets.FloatSlider(min=0.0,max=1.0,step=0.05,value=0.5,description='Location at the compartment: stimx',layout=Layout(width='50%'), continuous_update = False), 
                          tau1=widgets.FloatSlider(min=0.1,max=15.0,step=0.1,value=0.5,description='Synaptic rise time const: tau1',layout=Layout(width='50%'), continuous_update = False),
                          tau2=widgets.FloatSlider(min=15.0,max=100.0,step=1,value=20,description='Synaptic decay time const: tau2',layout=Layout(width='50%'), continuous_update = False),
                          glsoma=widgets.FloatSlider(min=0.0,max=0.5,step=0.01,value=0.03,description='Leak conductance, soma: glsoma',layout=Layout(width='50%'), continuous_update = False), 
                          gldend=widgets.FloatSlider(min=0.0,max=0.5,step=0.01,value=0.03,description='Leak conductance, dend: gldend',layout=Layout(width='50%'), continuous_update = False), 
                          Ra=widgets.FloatSlider(min=10.0,max=1000.0,step=10,value=100,description='Axial resistance: Ra',layout=Layout(width='50%'), continuous_update = False),
                          cm=widgets.FloatSlider(min=0.1,max=10.0,step=0.1,value=1.0,description='Membrane capacitance: cm',layout=Layout(width='50%'), continuous_update = False))


interactive(children=(FloatSlider(value=0.12, continuous_update=False, description='Amplitude of stimulus: amp…

The upper panel shows the membrane potential time course of the presynaptic (stimulated with current clamp) neuron, and the lower panel shows the membrane potential time course of the postsynaptic neuron. The presynaptic neuron is always stimulated at the soma, while the location of the synapse on the postsynaptic neuron can be varied.

#Question 1: Make the presynaptic neuron fire repetitively by making the duration of the current clamp longer. Changing the weight of the synapse (amp2), can you make the post-synaptic neuron fire to several first ones (but not all) of the synaptic inputs?

#***Question 2***: Can you make the post-synaptic neuron fire not after first synaptic input but after the second one? <i>Hint: increase the rise time constant of the synapse, and set the synapse further away from the soma. Slightly increase the membrane capacitances.</i>